# Quant ETF Features 📈
This notebook shows how to develop features on ETF ohlcv data

## 1. Setup

In [1]:
import os,sys
import duckdb
from pathlib import Path
import pandas as pd
import json

In [2]:
PROJECT_ROOT = Path.cwd().parents[0]

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print(f"Project Root: {PROJECT_ROOT}")

Project Root: C:\Users\luyanda\workspace\QuantTrade


In [3]:
DB_DAILY = PROJECT_ROOT / "data" / "processed" / "dolt" / "stocks.duckdb"
print(f"DB_DAILY: {DB_DAILY}")
con_daily = duckdb.connect(str(DB_DAILY))
tables = [t[0] for t in con_daily.execute("SHOW TABLES").fetchall()]
print("📋 Tables:", tables)

DB_DAILY: C:\Users\luyanda\workspace\QuantTrade\data\processed\dolt\stocks.duckdb
📋 Tables: ['dividend', 'ohlcv', 'split', 'symbol']


In [4]:
DB_MINUTE = PROJECT_ROOT / "data" / "processed" / "alpaca" / "price_minute_alpaca.duckdb"
print(f"DB_MINUTE: {DB_MINUTE}")
con_minute = duckdb.connect(str(DB_MINUTE))
tables = [t[0] for t in con_minute.execute("SHOW TABLES").fetchall()]
print("📋 Tables:", tables)

DB_MINUTE: C:\Users\luyanda\workspace\QuantTrade\data\processed\alpaca\price_minute_alpaca.duckdb
📋 Tables: ['alpaca_minute']


In [34]:
ETFS = ["SPY", "QQQ"]
CHARTS_DIR = PROJECT_ROOT / "charts" 

## 2. Helpers

In [6]:
def to_bt_daily_duckdb(con, symbol: str,
                       table: str = "ohlcv",
                       date_col: str = "date",
                       symbol_col: str = "act_symbol",
                       start=None, end=None):
    """
    Build clean daily OHLCV from DuckDB:
    - tz-naive DatetimeIndex named 'datetime'
    - one row per calendar day per symbol
    - O=first, H=max, L=min, C=last, V=sum
    - optional start/end (inclusive) on the date column
    """
    where = [f"{symbol_col} = ?"]
    params = [symbol]

    if start is not None:
        where.append(f"{date_col} >= ?")
        params.append(pd.to_datetime(start).date())
    if end is not None:
        where.append(f"{date_col} <= ?")  # inclusive end
        params.append(pd.to_datetime(end).date())

    where_sql = "WHERE " + " AND ".join(where)

    # If there are multiple rows per (symbol, date), aggregate to true daily bars.
    q = f"""
    WITH d AS (
      SELECT
        {symbol_col} AS symbol,
        CAST({date_col} AS TIMESTAMP) AS datetime,
        first(open)  AS open,     -- 'first' within the day (if duplicates)
        max(high)    AS high,
        min(low)     AS low,
        last(close)  AS close,    -- 'last' within the day (if duplicates)
        sum(volume)  AS volume
      FROM {table}
      {where_sql}
      GROUP BY 1, 2
    )
    SELECT symbol, datetime, open, high, low, close, volume
    FROM d
    WHERE open IS NOT NULL AND high IS NOT NULL AND low IS NOT NULL AND close IS NOT NULL
    ORDER BY datetime
    """

    df = con.execute(q, params).df()
    if df.empty:
        # Return a properly-shaped empty frame
        return pd.DataFrame(columns=["open","high","low","close","volume"]).astype({
            "open":"float64","high":"float64","low":"float64","close":"float64","volume":"float64"
        }).set_index(pd.DatetimeIndex([], name="datetime"))

    df["datetime"] = pd.to_datetime(df["datetime"])  # tz-naive at midnight
    df.columns = [c.lower() for c in df.columns]
    out = df.drop(columns=["symbol"]).set_index("datetime").sort_index()
    return out


In [7]:
def to_bt_minute_duckdb(con, table, symbol):
    """
    Build *true* 1-minute OHLCV bars for a single symbol using DuckDB and return
    a tidy Pandas DataFrame indexed by tz-naive 'datetime'.

    Requirements (columns in `table`):
      - timestamp (TIMESTAMP)
      - open, high, low, close (NUMERIC)
      - volume (NUMERIC)
      - trade_count (optional but used below)
      - vwap (optional but used below)

    How it works:
      - date_trunc('minute', timestamp) bins rows into 1-minute buckets.
      - open  = first price in the minute  → min_by(open,  timestamp)
      - high  = max price in the minute    → max(high)
      - low   = min price in the minute    → min(low)
      - close = last price in the minute   → max_by(close, timestamp)
      - volume      = sum(volume) across the minute
      - trade_count = sum(trade_count) across the minute
      - vwap        = volume-weighted average across the minute:
                      sum(vwap * volume) / sum(volume)
                      (this correctly aggregates sub-minute VWAPs if each row’s
                       vwap is itself volume-weighted for that sub-interval)

    Notes:
      - GROUP BY 1 groups by the first selected column (datetime).
      - ORDER BY 1 sorts by that same column.
      - If your source table lacks `trade_count` or `vwap`, remove those lines.
    """
    
    q = f"""
    SELECT
      date_trunc('minute', timestamp) AS datetime,
      min_by(open, timestamp)  AS open,
      max(high)                 AS high,
      min(low)                  AS low,
      max_by(close, timestamp)  AS close,
      sum(volume)               AS volume,
      sum(trade_count)          AS trade_count,
      CASE WHEN sum(volume) > 0
           THEN sum(vwap * volume) / sum(volume)
           ELSE NULL END        AS vwap
    FROM {table}
    WHERE symbol = ?
    GROUP BY 1
    ORDER BY 1
    """
    df = con.execute(q, [symbol]).df()
    df["datetime"] = pd.to_datetime(df["datetime"])
    return df.set_index("datetime")

In [8]:
def add_mas_duckdb(
    data_by_sym: dict[str, pd.DataFrame],
    con: duckdb.DuckDBPyConnection,
    windows: list[int],
    *,
    price_col: str = "close",
    prefix: str = "ma",          # columns like ma20, ma50, ...
) -> dict[str, pd.DataFrame]:
    """
    Add simple moving averages (SMA) to each df in a {symbol: DataFrame} dict using DuckDB.

    Assumptions:
      - Each DataFrame is indexed by a datetime-like index (e.g., 'datetime').
      - Each DataFrame has a 'close' column (override via price_col).
      - All original columns are preserved; new columns 'ma{window}' are appended.

    Why this is fast:
      - Concats all symbols into one table, computes all MAs in a single DuckDB query
        using window functions, then splits back to dict.
    """
    if not data_by_sym:
        return data_by_sym
    if not windows:
        return data_by_sym
    if any(w <= 0 for w in windows):
        raise ValueError("All moving-average windows must be positive integers.")

    # 1) Stack all symbols into one frame with a 'symbol' column + 'datetime' column
    frames = []
    for sym, df in data_by_sym.items():
        if df.empty:
            continue
        if price_col not in df.columns:
            raise KeyError(f"Expected column '{price_col}' in DataFrame for {sym}.")
        tmp = df.sort_index().reset_index()
        # ensure the first column (former index) is named 'datetime'
        tmp = tmp.rename(columns={tmp.columns[0]: "datetime"})
        tmp["symbol"] = sym
        frames.append(tmp)

    if not frames:
        return data_by_sym

    all_bars = pd.concat(frames, ignore_index=True)

    # 2) Register and build dynamic MA columns
    view_name = "_bars_for_ma"
    con.register(view_name, all_bars)

    ma_cols_sql = ",\n".join(
        [
            f"avg({price_col}) OVER (PARTITION BY symbol "
            f"ORDER BY datetime ROWS BETWEEN {w-1} PRECEDING AND CURRENT ROW) AS {prefix}{w}"
            for w in windows
        ]
    )

    sql = f"""
    SELECT
      *,
      {ma_cols_sql}
    FROM {view_name}
    ORDER BY symbol, datetime
    """

    result = con.execute(sql).df()
    # Clean up temp view (optional)
    con.unregister(view_name)

    # 3) Split back into dict, set index to datetime
    result["datetime"] = pd.to_datetime(result["datetime"])
    out: dict[str, pd.DataFrame] = {}
    for sym, g in result.groupby("symbol", sort=False):
        g = g.drop(columns=["symbol"]).set_index("datetime").sort_index()
        out[sym] = g

    return out


## 3. Data Ingestion

In [9]:
# --- Ingest latest daily data ---
daily_data = {
    sym: to_bt_daily_duckdb(con_daily, sym, table="ohlcv", date_col="date", symbol_col="act_symbol")
    for sym in ETFS
}

for symbol in ETFS:
    print(daily_data[symbol].tail(1))


              open    high     low   close      volume
datetime                                              
2025-08-12  638.29  642.85  636.79  642.69  64821798.0
              open    high     low   close      volume
datetime                                              
2025-08-12  575.16  580.35  572.49  580.05  42271441.0


In [10]:
# --- Ingest latest minute-level data ---
minute_data = {sym: to_bt_minute_duckdb(con_minute, "alpaca_minute", sym) for sym in ETFS}

for symbol in ETFS:
    print(minute_data[symbol].tail(1))

                       open    high     low   close  volume  trade_count  \
datetime                                                                   
2025-08-13 14:36:00  644.34  644.34  644.34  644.34   100.0          1.0   

                       vwap  
datetime                     
2025-08-13 14:36:00  644.34  
                       open    high     low   close  volume  trade_count  \
datetime                                                                   
2025-08-13 14:55:00  582.45  582.45  582.45  582.45   340.0          2.0   

                       vwap  
datetime                     
2025-08-13 14:55:00  582.45  


## 4. Data Quality Checks

U.S. Market (SPY, QQQ)
Assuming regular NYSE/Nasdaq trading hours:

| **Session**     | **Hours (ET)**   | **Duration** |
| --------------- | ---------------- | ------------ |
| Regular session | 09:30 – 16:00 ET | 6.5 hours    |
|                 |                  | 390 minutes  |

Expect around 390 rows per ETF

In [11]:
for symbol in ETFS:
    df = minute_data[symbol]
    print(f"\n🔍 {symbol}")
    print(f"  • Rows: {len(df)}")
    print(f"  • Date Range: {df.index.min().date()} → {df.index.max().date()}")
    print(f"  • Timezone-aware: {df.index.tz is not None}")
    # print(f"  • Missing 'close': {df['close'].isna().sum()}")

    # --- Drop timezone if needed ---
    df = df.copy()
    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)

    # --- Identify all available intraday dates ---
    df["date"] = df.index.normalize()
    available_dates = df["date"].unique()

    # --- Construct full expected range (business days) ---
    expected_dates = pd.date_range(
        start=df.index.min().normalize(),
        end=df.index.max().normalize(),
        freq='B'
    )

    # --- Missing trading days entirely ---
    missing_dates = sorted(set(expected_dates) - set(available_dates))
    print(f"  • Missing Intraday Dates: {len(missing_dates)}")
    # if missing_dates:
    #     print("    Example:", missing_dates[:5])

    # --- Check for partial trading days (fewer than 390 rows) ---
    counts = df.groupby("date").size()
    partial_days = counts[counts < 390]
    print(f"  • Partial Intraday Days (<390 rows): {len(partial_days)}")
    # if not partial_days.empty:
    #     print("    Example:", partial_days.head())



🔍 SPY
  • Rows: 192320
  • Date Range: 2023-08-09 → 2025-08-13
  • Timezone-aware: False
  • Missing Intraday Dates: 22
  • Partial Intraday Days (<390 rows): 279

🔍 QQQ
  • Rows: 187087
  • Date Range: 2023-08-09 → 2025-08-13
  • Timezone-aware: False
  • Missing Intraday Dates: 22
  • Partial Intraday Days (<390 rows): 306


## 5. Feature Engineering

In [13]:
# # Example: add 20/50/200 SMAs to your daily_data dict using the stocks connection
# daily_data_ma = add_mas_duckdb(daily_data, con_daily, windows=[20, 50, 200])
# print(daily_data_ma["SPY"].tail(1))
# print(daily_data_ma["QQQ"].tail(1))

# Or for minute bars (same API). E.g., minute_data already built via DuckDB:
minute_data_ma = add_mas_duckdb(minute_data, con_minute, windows=[20, 50,200], price_col="close")
print(minute_data_ma["SPY"].tail(1))
print(minute_data_ma["QQQ"].tail(1))

                       open    high     low   close  volume  trade_count  \
datetime                                                                   
2025-08-13 14:36:00  644.34  644.34  644.34  644.34   100.0          1.0   

                       vwap     ma20      ma50       ma200  
datetime                                                    
2025-08-13 14:36:00  644.34  642.645  642.4728  642.115025  
                       open    high     low   close  volume  trade_count  \
datetime                                                                   
2025-08-13 14:55:00  582.45  582.45  582.45  582.45   340.0          2.0   

                       vwap      ma20     ma50       ma200  
datetime                                                    
2025-08-13 14:55:00  582.45  580.5895  580.177  579.589975  


In [39]:
def render_lightweight_chart(
    df: pd.DataFrame,
    *,
    symbol: str = "SYMBOL",
    out_html: str | Path = "chart.html",
    theme: str = "dark",
    height: int = 700,
    title: str | None = None,
    # ---- indicators (dynamic per timeframe) ----
    ma_windows: list[int] | None = None,   # e.g., [20, 50, 200]
    # ---- timeframe switcher ----
    timeframes: list[str] = ("1m","5m","15m","1h","1d"),
    default_tf: str = "1m",
    digits: int = 2,
    # ---- watermark ----
    watermark_text: str | None = None,     # e.g., "SPY — {tf}" (use {tf} to show current timeframe)
    watermark_opacity: float = 0.08,
    watermark_angle_deg: float = -18.0,
    watermark_font_css: str | None = None, # e.g., "900 11vw/1 -apple-system, Segoe UI, Roboto, sans-serif"
    watermark_color: str | None = None,    # default: auto (light/dark aware)
):
    """
    Lightweight Charts with:
      - Candlesticks + Volume
      - Timeframe switcher (client-side aggregation): 1m/5m/15m/1h/1d
      - Dynamic SMA lines recomputed on the selected timeframe (ma_windows)
      - Optional centered watermark (uses {tf} placeholder to reflect selected timeframe)

    df index: datetime-like (naive or tz-aware)
    required columns: open, high, low, close, volume
    """
    core = {"open","high","low","close","volume"}
    for c in core:
        if c not in df.columns:
            raise KeyError(f"Missing required column '{c}'")

    # Prepare UTC timestamps
    tmp = df.copy()
    idx = pd.to_datetime(tmp.index)
    tmp.index = (idx.tz_convert("UTC") if idx.tz is not None else idx.tz_localize("UTC"))
    tmp = tmp.dropna(subset=["open","high","low","close"])
    if tmp.empty:
        raise ValueError("No rows with complete OHLC to plot.")

    def to_ts(x): return int(pd.Timestamp(x).timestamp())

    candles = [
        {"time": to_ts(t), "open": float(r.open), "high": float(r.high),
         "low": float(r.low), "close": float(r.close)}
        for t, r in tmp[["open","high","low","close"]].iterrows()
    ]
    volumes = []
    for t, r in tmp[["open","close","volume"]].iterrows():
        up = (r.close >= r.open)
        volumes.append({
            "time": to_ts(t),
            "value": float(r.volume or 0.0),
            "color": "#26a69a" if up else "#ef5350",
        })

    # Palette for MA lines
    ma_windows = ma_windows or []
    ma_names = [f"ma{w}" for w in ma_windows]
    palette = ["#ff9800","#42a5f5","#ab47bc","#26a69a","#ec407a",
               "#8d6e63","#66bb6a","#ffa726","#29b6f6","#ef5350"]
    color_map = {name: palette[i % len(palette)] for i, name in enumerate(ma_names)}

    title = title or f"{symbol} • Lightweight Charts"

    # Watermark defaults
    wm_color = watermark_color or ("#ffffff" if theme == "dark" else "#111827")
    wm_font = watermark_font_css or "900 11vw/1 -apple-system, Segoe UI, Roboto, sans-serif"
    wm_text = watermark_text or ""  # empty means hidden
    wm_style_display = "" if wm_text else "display:none;"

    html = f"""<!doctype html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>{title}</title>
  <script src="https://unpkg.com/lightweight-charts@4.2.1/dist/lightweight-charts.standalone.production.js"></script>
  <style>
    html, body {{ margin:0; padding:0; background:{("#0e1117" if theme=="dark" else "#ffffff")}; }}
    #wrap {{ position:relative; }}
    #toolbar {{
      display:flex; gap:8px; padding:8px 12px;
      background:{("rgba(14,17,23,0.65)" if theme=="dark" else "rgba(255,255,255,0.9)")};
      position:sticky; top:0; z-index:10;
      font:13px -apple-system, Segoe UI, Roboto, sans-serif;
      color:{("#e5e7eb" if theme=="dark" else "#111827")};
      backdrop-filter: blur(4px);
    }}
    .btn {{
      padding:4px 10px; border-radius:6px; cursor:pointer; user-select:none;
      border:1px solid {("#374151" if theme=="dark" else "#d1d5db")};
      background:{("#111827" if theme=="dark" else "#ffffff")};
    }}
    .btn.active {{
      background:{("#1f2937" if theme=="dark" else "#f3f4f6")};
      border-color:{("#6b7280" if theme=="dark" else "#9ca3af")};
      font-weight:600;
    }}
    #chart {{ height:{height}px; }}
    .legend {{
      position:absolute; left:12px; top:54px; padding:6px 8px;
      background:{("rgba(14,17,23,0.65)" if theme=="dark" else "rgba(255,255,255,0.9)")};
      border-radius:8px; font:12px/1.25 -apple-system, Segoe UI, Roboto, sans-serif;
      color:{("#e5e7eb" if theme=="dark" else "#111827")}; box-shadow:0 2px 6px rgba(0,0,0,.15);
      z-index: 5;
    }}
    /* ✅ Watermark overlay */
    .watermark {{
      position:absolute; inset:0; display:flex; align-items:center; justify-content:center;
      pointer-events:none; user-select:none; z-index: 0;
      transform: rotate({watermark_angle_deg}deg);
      opacity: {watermark_opacity};
      color: {wm_color};
      font: {wm_font};
      letter-spacing: 0.08em;
      text-transform: uppercase;
      mix-blend-mode: {"screen" if theme=="dark" else "multiply"};
    }}
  </style>
</head>
<body>
  <div id="wrap">
    <div id="toolbar"></div>
    <div id="chart"></div>
    <div id="legend" class="legend"></div>

    <!-- ✅ Watermark element -->
    <div id="wm" class="watermark" style="{wm_style_display}"></div>
  </div>
  <script>
    // ---- Data from Python ----
    const baseCandles = {json.dumps(candles)};
    const baseVolumes = {json.dumps(volumes)};
    const TF_OPTIONS   = {json.dumps(list(timeframes))};
    let   currentTF    = "{default_tf}";
    const MA_WINDOWS   = {json.dumps(ma_windows)};
    const MA_COLORS    = {json.dumps(color_map)};
    const DIGITS       = {digits};
    const SYMBOL       = {json.dumps(symbol)};
    const WM_TEMPLATE  = {json.dumps(wm_text)}; // may contain "{'{tf}'}"

    // ---- Helpers ----
    function tfToMinutes(tf) {{
      const m = tf.toLowerCase();
      if (m.endsWith('m')) return parseInt(m);
      if (m.endsWith('h')) return parseInt(m) * 60;
      if (m.endsWith('d')) return 'day';
      return 1;
    }}

    function bucketTimeSec(ts, tf) {{
      const kind = tfToMinutes(tf);
      if (kind === 'day') {{
        const d = new Date(ts * 1000);
        return Date.UTC(d.getUTCFullYear(), d.getUTCMonth(), d.getUTCDate()) / 1000;
      }} else {{
        const mins   = Math.floor(ts / 60);
        const bstart = mins - (mins % kind);
        return bstart * 60;
      }}
    }}

    function aggregate(candles, volumes, tf) {{
      if (!candles.length) return {{candles: [], volumes: []}};
      const map = new Map();
      for (let i = 0; i < candles.length; i++) {{
        const c = candles[i];
        const ts = bucketTimeSec(c.time, tf);
        let b = map.get(ts);
        if (!b) {{
          b = {{ time: ts, open: c.open, high: c.high, low: c.low, close: c.close, volume: 0 }};
          map.set(ts, b);
        }} else {{
          b.high = Math.max(b.high, c.high);
          b.low  = Math.min(b.low,  c.low);
          b.close = c.close;
        }}
      }}
      for (let i = 0; i < volumes.length; i++) {{
        const v = volumes[i];
        const ts = bucketTimeSec(v.time, tf);
        const b = map.get(ts);
        if (b) b.volume += (v.value || 0);
      }}
      const outCandles = Array.from(map.values()).sort((a,b)=>a.time-b.time);
      const outVolumes = outCandles.map(b => ({{
        time: b.time, value: b.volume, color: (b.close >= b.open) ? '#26a69a' : '#ef5350'
      }}));
      return {{candles: outCandles, volumes: outVolumes}};
    }}

    function computeSMA(candles, window) {{
      const out = [];
      let sum = 0;
      const w = window;
      for (let i=0; i<candles.length; i++) {{
        const c = candles[i];
        sum += c.close;
        if (i >= w) sum -= candles[i - w].close;
        const val = (i >= w - 1) ? (sum / w) : null;
        out.push({{ time: c.time, value: (val===null? null : +val) }});
      }}
      return out;
    }}

    // ---- UI ----
    const toolbar = document.getElementById('toolbar');
    for (const tf of TF_OPTIONS) {{
      const btn = document.createElement('div');
      btn.className = 'btn' + (tf === currentTF ? ' active' : '');
      btn.textContent = tf;
      btn.onclick = () => setTimeframe(tf);
      toolbar.appendChild(btn);
    }}

    // ---- Chart ----
    const chart = LightweightCharts.createChart(document.getElementById('chart'), {{
      layout: {{
        background: {{ type:'solid', color:'{("#0e1117" if theme=="dark" else "#ffffff")}' }},
        textColor: '{("#d1d5db" if theme=="dark" else "#111827")}'
      }},
      grid: {{
        vertLines: {{ color:'{("#1f2937" if theme=="dark" else "#e5e7eb")}' }},
        horzLines: {{ color:'{("#1f2937" if theme=="dark" else "#e5e7eb")}' }}
      }},
      timeScale: {{ timeVisible:true, secondsVisible:false, rightOffset:6, barSpacing:6 }},
      rightPriceScale: {{ borderVisible:false }},
      crosshair: {{ mode: 0 }}
    }});

    const candleSeries = chart.addCandlestickSeries({{
      upColor:'#26a69a', downColor:'#ef5350',
      borderUpColor:'#26a69a', borderDownColor:'#ef5350',
      wickUpColor:'#26a69a', wickDownColor:'#ef5350'
    }});

    const volumeSeries = chart.addHistogramSeries({{
      priceScaleId:'vol', priceFormat:{{ type:'volume' }},
      priceLineVisible:false, base:0, scaleMargins:{{ top:0.8, bottom:0 }}
    }});
    chart.priceScale('vol').applyOptions({{ scaleMargins: {{ top: 0.8, bottom: 0 }} }});

    // MA line series
    const maSeries = {{}};
    for (const w of MA_WINDOWS) {{
      const name = 'ma' + w;
      maSeries[name] = chart.addLineSeries({{
        color: MA_COLORS[name] || '#888', lineWidth:2, priceLineVisible:false, title: name
      }});
    }}

    // Legend
    const legend = document.getElementById('legend');
    const fmt = n => (n==null || isNaN(n)) ? '' : Number(n).toFixed(DIGITS);
    const fmtVol = n => {{
      if (n==null || isNaN(n)) return '';
      if (n >= 1e9) return (n/1e9).toFixed(2)+'B';
      if (n >= 1e6) return (n/1e6).toFixed(2)+'M';
      if (n >= 1e3) return (n/1e3).toFixed(2)+'K';
      return Number(n).toFixed(0);
    }};

    function updateLegend(param) {{
      let c = param.seriesData ? param.seriesData.get(candleSeries) : null;
      let v = param.seriesData ? param.seriesData.get(volumeSeries) : null;
      if (!c) {{
        const lastIdx = currentCandles.length - 1;
        c = currentCandles[lastIdx];
        v = currentVolumes[lastIdx];
      }}
      const ts = c.time;
      const tstr = new Date(ts*1000).toISOString().replace('T',' ').slice(0,19) + ' UTC';
      let html = '';
      html += `<span class="row sym">${{SYMBOL}}</span>`;
      html += `<span class="row">${{tstr}}</span>`;
      html += `<span class="row">O:${{fmt(c.open)}} H:${{fmt(c.high)}} L:${{fmt(c.low)}} C:${{fmt(c.close)}}</span>`;
      if (v && v.value != null) html += `<span class="row">Vol:${{fmtVol(v.value)}}</span>`;
      for (const [name, series] of Object.entries(maSeries)) {{
        const sd = param.seriesData ? param.seriesData.get(series) : null;
        const val = sd ? sd.value : null;
        const col = MA_COLORS[name] || '#888';
        html += `<span class="row"><span class="dot" style="background:${{col}}"></span>${{name}}: ${{fmt(val)}}</span>`;
      }}
      legend.innerHTML = html;
    }}

    chart.subscribeCrosshairMove(updateLegend);

    // ---- Watermark updater ----
    const wmEl = document.getElementById('wm');
    function setWatermarkText(tf) {{
      if (!WM_TEMPLATE || WM_TEMPLATE.length === 0) return;
      const txt = WM_TEMPLATE.replace('{{tf}}', tf);
      wmEl.textContent = txt;
      wmEl.style.display = '';
    }}

    // ---- Apply timeframe ----
    let currentCandles = [];
    let currentVolumes = [];
    function setTimeframe(tf) {{
      currentTF = tf;
      const agg = aggregate(baseCandles, baseVolumes, tf);
      currentCandles = agg.candles;
      currentVolumes = agg.volumes;
      candleSeries.setData(currentCandles);
      volumeSeries.setData(currentVolumes);

      for (const w of MA_WINDOWS) {{
        const name = 'ma' + w;
        const line = computeSMA(currentCandles, w);
        maSeries[name].setData(line);
      }}

      for (const el of document.querySelectorAll('#toolbar .btn')) {{
        el.classList.toggle('active', el.textContent === tf);
      }}

      setWatermarkText(tf);
      chart.timeScale().fitContent();

      if (currentCandles.length) {{
        updateLegend({{ seriesData: new Map([[candleSeries, currentCandles[currentCandles.length-1]], [volumeSeries, currentVolumes[currentVolumes.length-1]]]) }});
      }}
    }}

    // Initial render
    setTimeframe(currentTF);

    // Responsive
    new ResizeObserver(e => {{
      const w = e[0].contentRect.width;
      chart.applyOptions({{ width: Math.max(320, Math.floor(w)) }});
    }}).observe(document.getElementById('wrap'));
  </script>
</body>
</html>"""

    out_html = Path(out_html)
    out_html.parent.mkdir(parents=True, exist_ok=True)
    out_html.write_text(html, encoding="utf-8")
    return out_html


In [ ]:
render_lightweight_chart(
    minute_data["SPY"],
    symbol="SPY",
    out_html=CHARTS_DIR/"spy_20ma50ma200ma.html",
    ma_windows=[20, 50, 200],
    timeframes=["1m","5m","15m","1h","1d"],
    default_tf="5m",
    watermark_text="SPY — {tf}",
    watermark_opacity=0.07,
)


WindowsPath('C:/Users/luyanda/workspace/QuantTrade/charts/spy_tf_wm.html')

In [ ]:
render_lightweight_chart(
    minute_data["QQQ"],
    symbol="QQQ",
    out_html=CHARTS_DIR/"qqq_20ma50ma200ma.html",
    ma_windows=[20, 50, 200],
    timeframes=["1m","5m","15m","1h","1d"],
    default_tf="5m",
    watermark_text="QQQ — {tf}",
    watermark_opacity=0.07,
)


WindowsPath('C:/Users/luyanda/workspace/QuantTrade/charts/qqq_20ma50ma200ma.html')